# Scraping Weatherspark.com

## 1. Web scraping Weatherspark.

In [27]:
#Importando las librerías necesarias.

import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [28]:
#Adjudicando el driver correcto para nuestro navegador, muy importante.

driver = webdriver.Chrome()

In [29]:
#Getting url.

driver.get('https://es.weatherspark.com/')

In [30]:
#Asignamos la caja de búsqueda al elemento correspondiente del site.

query_box = driver.find_element_by_class_name('LiveSearch-FrontPage-field')

In [31]:
#Comprobamos que nos devuelva un WebElement.

query_box

<selenium.webdriver.remote.webelement.WebElement (session="2a6121a70ba5f6fd24af1541a478fac8", element="2e8873d8-df08-4f89-99c1-9d93c2218f38")>

In [32]:
#Haciendo click en el search box.

query_box.click()

In [33]:
#Buscando con el keyword Madrid.

query_box.send_keys('Madrid')

In [34]:
#Simulando la tecla enter ya que el botón de búsqueda nos arrojaba fallos.

query_box.send_keys(Keys.ENTER)

In [35]:
results = driver.find_element_by_id("Report-Body")

In [36]:
#Resultados del query.

results_text = results.text

In [37]:
#Eliminamos los saltos de línea.

results_stripped = results_text.replace('\n','')

In [38]:
#Nuestros resultados para Madrid. Ahora nos tocará buscar las secciones del reporte meteorológico que nos interesen y
#seleccionarlas mediante regex.

results_stripped

'Anuncio publicitarioEl clima promedio en Madrid EspañaEn Madrid, los veranos son cortos, muy caliente, secos y mayormente despejados y los inviernos son muy frío y parcialmente nublados. Durante el transcurso del año, la temperatura generalmente varía de 0 °C a 33 °C y rara vez baja a menos de -5 °C o sube a más de 37 °C.En base a la puntuación de playa/piscina, la mejor época del año para visitar Madrid para las actividades de calor es desde finales de junio hasta finales de agostoAnuncio publicitarioResumen del climafríafrescacómodamuy calientefrescafríaene.feb.mar.abr.may.jun.jul.ago.sept.oct.nov.dic.AhoraAhora88 %88 %48 %48 %despejadonubladoprecipitación: 48 mmprecipitación: 48 mm6 mm6 mmbochornoso: 0 %bochornoso: 0 %0 %0 %secosecopuntuación de playa/piscina: 8.6puntuación de playa/piscina: 8.60.00.0Haga clic en cada gráfico para ver más informaciónAnuncio publicitarioTemperaturaLa temporada calurosa dura 2,9 meses, del 15 de junio al 10 de septiembre, y la temperatura máxima prom

In [39]:
#Comprobamos que nuestro primer filtro mediante regex captura el elemento que realmente buscamos.

import re

temporada = re.findall("(?=La temporada calurosa).*?(?=Temperatura máxima)", results_stripped)
temporada

['La temporada calurosa dura 2,9 meses, del 15 de junio al 10 de septiembre, y la temperatura máxima promedio diaria es más de 29 °C. El día más caluroso del año es el 23 de julio, con una temperatura máxima promedio de 33 °C y una temperatura mínima promedio de 18 °C.La temporada fresca dura 3,7 meses, del 14 de noviembre al 5 de marzo, y la temperatura máxima promedio diaria es menos de 15 °C. El día más frío del año es el 15 de enero, con una temperatura mínima promedio de 0 °C y máxima promedio de 10 °C.']

In [40]:
#Repetimos el mismo procedimiento para cada sección de interés. Asignar la búsqueda por regex a una variable nos facilitará
#crear una función para hacer el web scraping y generar un diccionario fácilmente.

nubosidad = re.findall("(?=La parte más despejada).*?(?=Categorías de nubosidad)", results_stripped)
nubosidad

['La parte más despejada del año en Madrid comienza aproximadamente el 11 de junio; dura 3,0 meses y se termina aproximadamente el 12 de septiembre. El 21 de julio, el día más despejado del año, el cielo está despejado, mayormente despejado o parcialmente nublado el 88 % del tiempo y nublado o mayormente nublado el 12 % del tiempo.La parte más nublada del año comienza aproximadamente el 12 de septiembre; dura 9,0 meses y se termina aproximadamente el 11 de junio. El 16 de diciembre, el día más nublado del año, el cielo está nublado o mayormente nublado el 52 % del tiempo y despejado, mayormente despejado o parcialmente nublado el 48 % del tiempo.']

In [41]:
precipitaciones = re.findall("(?=La temporada más mojada).*?(?=Probabilidad diaria de precipitación)", results_stripped)
precipitaciones

['La temporada más mojada dura 8,4 meses, de 1 de octubre a 14 de junio, con una probabilidad de más del 14 % de que cierto día será un día mojado. La probabilidad máxima de un día mojado es del 23 % el 29 de abril.La temporada más seca dura 3,6 meses, del 14 de junio al 1 de octubre. La probabilidad mínima de un día mojado es del 4 % el 19 de julio.Entre los días mojados, distinguimos entre los que tienen solamente lluvia, solamente nieve o una combinación de las dos. En base a esta categorización, el tipo más común de precipitación durante el año es solo lluvia, con una probabilidad máxima del 23 % el 29 de abril.']

In [42]:
lluvia = re.findall("(?=La temporada de lluvia).*?(?=Precipitación de lluvia mensual promedio)", results_stripped)
lluvia

['La temporada de lluvia dura 9,9 meses, del 4 de septiembre al 2 de julio, con un intervalo móvil de 31 días de lluvia de por lo menos 13 milímetros. La mayoría de la lluvia cae durante los 31 días centrados alrededor del 27 de octubre, con una acumulación total promedio de 48 milímetros.El periodo del año sin lluvia dura 2,1 meses, del 2 de julio al 4 de septiembre. La fecha aproximada con la menor cantidad de lluvia es el 31 de julio, con una acumulación total promedio de 6 milímetros.']

In [43]:
turismo = re.findall("(?=La puntuación de turismo).*?(?=Puntuación de turismo)", results_stripped)
turismo

['La puntuación de turismo favorece los días despejados y sin lluvia con temperaturas percibidas entre 18 °C y 27 °C. En base a esta puntuación, la mejor época del año para visitar Madrid para las actividades turísticas generales a la intemperie es desde finales de mayo hasta principios de octubre, con una puntuación máxima en la segunda semana de septiembre.']

In [85]:
#Ahora que tenemos todas los componentes de la función testeados individualmente es hora de ponerlos a trabajar. Definimos
#la función para realizar el scraping de las localidades (en nuestro caso serán las capitales de provincia).

import time

def meteo(localidad):
    df_provincias 
    driver = webdriver.Chrome()
    driver.get('https://es.weatherspark.com/')
    query_box = driver.find_element_by_class_name('LiveSearch-FrontPage-field')
    time.sleep(1) #Incluimos una pausa de 1s para que el navegador tenga tiempo de cargar la información.
    query_box.click()
    query_box.send_keys(localidad + ' ' + 'ESPAÑA')
    query_box.send_keys(Keys.ENTER)
    time.sleep(1)
    results = driver.find_element_by_id("Report-Body")
    results_text = results.text
    results_stripped = results_text.replace('\n','')
    temporada = re.findall("(?=La temporada).*?(?=Temperatura máxima)", results_stripped)
    nubosidad = re.findall("(?=La parte más despejada).*?(?=Categorías de nubosidad)", results_stripped)
    precipitaciones = re.findall("(?=La temporada más mojada).*?(?=Probabilidad diaria de precipitación)", results_stripped)
    lluvia = re.findall("(?=La mayoría de la lluvia).*?(?=Precipitación de lluvia mensual promedio)", results_stripped)
    turismo = re.findall("(?=La puntuación de turismo).*?(?=Puntuación de turismo)", results_stripped)
    meteo_localidad = {"capital": localidad, "temporadas": temporada[0],  "nubosidad": nubosidad[0], "precipitaciones": precipitaciones[0], "lluvia": lluvia[0], "turismo": turismo[0]}
    return meteo_localidad #La función simplemente retorna un diccionario con la localidad y las características de la meteo. 

In [86]:
#Testeando la función.

meteo('GIRONA')

{'capital': 'GIRONA',
 'temporadas': 'La temporada calurosa dura 2,8 meses, del 18 de junio al 10 de septiembre, y la temperatura máxima promedio diaria es más de 27 °C. El día más caluroso del año es el 5 de agosto, con una temperatura máxima promedio de 31 °C y una temperatura mínima promedio de 18 °C.La temporada fresca dura 4,0 meses, del 16 de noviembre al 15 de marzo, y la temperatura máxima promedio diaria es menos de 17 °C. El día más frío del año es el 10 de enero, con una temperatura mínima promedio de 2 °C y máxima promedio de 13 °C.',
 'nubosidad': 'La parte más despejada del año en Girona comienza aproximadamente el 19 de junio; dura 2,5 meses y se termina aproximadamente el 5 de septiembre. El 20 de julio, el día más despejado del año, el cielo está despejado, mayormente despejado o parcialmente nublado el 82 % del tiempo y nublado o mayormente nublado el 18 % del tiempo.La parte más nublada del año comienza aproximadamente el 5 de septiembre; dura 9,5 meses y se termina 

## 2. Obteniendo los datos meteorológicos de cada provincia en un DF.

In [87]:
#Importamos el DF de provincias creado anteriormente para combinarlo con nuestro DF de meteorología.

import pandas as pd

df_provincias = pd.read_csv('provincias.csv')
df_provincias.head()

,provincia,altitud,distancia,desnivel,pendiente,coeficiente
0,A CORUÑA,505,6,436,7,117
1,ALBACETE,1402,12,782,6,174
2,ALICANTE,810,7,577,7,171
3,ALMERÍA,1670,18,1001,4,216
4,ASTURIAS,1163,11,818,7,226


In [88]:
#Añadimos una columna para la capital de provincia, que usaremos para obtener los datos meteorológicos.

df_provincias['capital'] = df_provincias['provincia']
df_provincias.head()

,provincia,altitud,distancia,desnivel,pendiente,coeficiente,capital
0,A CORUÑA,505,6,436,7,117,A CORUÑA
1,ALBACETE,1402,12,782,6,174,ALBACETE
2,ALICANTE,810,7,577,7,171,ALICANTE
3,ALMERÍA,1670,18,1001,4,216,ALMERÍA
4,ASTURIAS,1163,11,818,7,226,ASTURIAS


In [90]:
#Ya que algunas provincias no comparten el nombre con su capital necesitaremos cambiar esos valores.

df_provincias['capital'] = df_provincias['capital'].replace(['ÁLAVA'], 'VITORIA')
df_provincias['capital'] = df_provincias['capital'].replace(['ASTURIAS'], 'OVIEDO')
df_provincias['capital'] = df_provincias['capital'].replace(['CANTBRIA'], 'SANTANDER')
df_provincias['capital'] = df_provincias['capital'].replace(['GIPUZKOA'], 'SAN SEBASTIÁN')
df_provincias['capital'] = df_provincias['capital'].replace(['NAVARRA'], 'PAMPLONA')
df_provincias['capital'] = df_provincias['capital'].replace(['LA RIOJA'], 'LOGROÑO')
df_provincias['capital'] = df_provincias['capital'].replace(['BIZKAIA'], 'BILBAO')

In [91]:
#Creamos una lista con la columna de capitales para poder pasarla fácilmente a la función.

lista_capitales = df_provincias['capital'].tolist()
print(lista_capitales)

['A CORUÑA', 'ALBACETE', 'ALICANTE', 'ALMERÍA', 'OVIEDO', 'BADAJOZ', 'BARCELONA', 'BILBAO', 'BURGOS', 'CANTABRIA', 'CASTELLÓN', 'CEUTA', 'CIUDAD REAL', 'CUENCA', 'CÁCERES', 'CÁDIZ', 'CÓRDOBA', 'GERONA', 'SAN SEBASTIÁN', 'GRANADA', 'GUADALAJARA', 'HUELVA', 'HUESCA', 'JAÉN', 'LOGROÑO', 'LEÓN', 'LLEIDA', 'LUGO', 'MADRID', 'MURCIA', 'MÁLAGA', 'PAMPLONA', 'OURENSE', 'PALENCIA', 'PONTEVEDRA', 'SALAMANCA', 'SEGOVIA', 'SEVILLA', 'SORIA', 'TARRAGONA', 'TERUEL', 'TOLEDO', 'VALENCIA', 'ZAMORA', 'ZARAGOZA', 'VITORIA', 'ÁVILA']


In [96]:
#Comprobando que podamos pasar el diccionario generado por la función a un DF sin mayor inconveniente.

df_meteo = pd.DataFrame([meteo('A CORUÑA')])
df_meteo.head()

,capital,temporadas,nubosidad,precipitaciones,lluvia,turismo
0,A CORUÑA,"La temporada templada dura 3,1 meses, del 21 d...",La parte más despejada del año en La Coruña co...,"La temporada más mojada dura 7,8 meses, de 28 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...


In [97]:
#Pasando la función a la lista de capitales.

for i in lista_capitales[1:]:
    try:
        meteo_dict = meteo(i)
        df_meteo = df_meteo.append(meteo_dict, ignore_index=True)
    except:
        pass

In [100]:
df_meteo.head()

,capital,temporadas,nubosidad,precipitaciones,lluvia,turismo
0,A CORUÑA,"La temporada templada dura 3,1 meses, del 21 d...",La parte más despejada del año en La Coruña co...,"La temporada más mojada dura 7,8 meses, de 28 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
1,ALBACETE,"La temporada calurosa dura 2,8 meses, del 15 d...",La parte más despejada del año en Albacete com...,"La temporada más mojada dura 9,1 meses, de 11 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
2,ALICANTE,"La temporada calurosa dura 3,0 meses, del 20 d...",La parte más despejada del año en Alicante com...,"La temporada más mojada dura 8,6 meses, de 8 d...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
3,ALMERÍA,"La temporada calurosa dura 2,9 meses, del 20 d...",La parte más despejada del año en Almería comi...,"La temporada más mojada dura 7,6 meses, de 27 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
4,OVIEDO,"La temporada templada dura 2,9 meses, del 25 d...",La parte más despejada del año en Oviedo comie...,"La temporada más mojada dura 8,0 meses, de 26 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...


In [101]:
#Merging los dos dataframes:

df_meteo_provincias = pd.merge(df_provincias, df_meteo, on='capital')
df_meteo_provincias.head()

,provincia,altitud,distancia,desnivel,pendiente,coeficiente,capital,temporadas,nubosidad,precipitaciones,lluvia,turismo
0,A CORUÑA,505,6,436,7,117,A CORUÑA,"La temporada templada dura 3,1 meses, del 21 d...",La parte más despejada del año en La Coruña co...,"La temporada más mojada dura 7,8 meses, de 28 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
1,ALBACETE,1402,12,782,6,174,ALBACETE,"La temporada calurosa dura 2,8 meses, del 15 d...",La parte más despejada del año en Albacete com...,"La temporada más mojada dura 9,1 meses, de 11 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
2,ALICANTE,810,7,577,7,171,ALICANTE,"La temporada calurosa dura 3,0 meses, del 20 d...",La parte más despejada del año en Alicante com...,"La temporada más mojada dura 8,6 meses, de 8 d...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
3,ALMERÍA,1670,18,1001,4,216,ALMERÍA,"La temporada calurosa dura 2,9 meses, del 20 d...",La parte más despejada del año en Almería comi...,"La temporada más mojada dura 7,6 meses, de 27 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...
4,ASTURIAS,1163,11,818,7,226,OVIEDO,"La temporada templada dura 2,9 meses, del 25 d...",La parte más despejada del año en Oviedo comie...,"La temporada más mojada dura 8,0 meses, de 26 ...",La mayoría de la lluvia cae durante los 31 día...,La puntuación de turismo favorece los días des...


In [1]:
#Finalmente exportamos el dataframe.

df_meteo_provincias.to_csv('meteo_provincias.csv', index = False)

NameError: name 'df_meteo_provincias' is not defined

In [ ]:
###FINALIZADO###